<h1 style="text-align:center">My House Price Prediction Approach</h1>

<div style="text-align:center;"><img src="https://images.unsplash.com/photo-1516156008625-3a9d6067fab5?ixlib=rb-1.2.1&auto=format&fit=crop&w=1500&q=80" /></div>

# Overview

**Context:** 
> Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.   
With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

**About the Data:**

*File descriptions*
* train.csv - the training set
* test.csv - the test set
* data_description.txt - full description of each column, originally prepared by Dean De Cock but lightly edited to match the column names used here
* sample_submission.csv - a benchmark submission from a linear regression on year and month of sale, lot square footage, and number of bedrooms


*Data fields*
Here's a brief version of what you'll find in the data description file.

* SalePrice - the property's sale price in dollars. This is the target variable that you're trying to predict.
* MSSubClass: The building class
* MSZoning: The general zoning classification
* LotFrontage: Linear feet of street connected to property
* LotArea: Lot size in square feet
* Street: Type of road access
* Alley: Type of alley access
* LotShape: General shape of property
* LandContour: Flatness of the property
* Utilities: Type of utilities available
* LotConfig: Lot configuration
* LandSlope: Slope of property
* Neighborhood: Physical locations within Ames city limits
* Condition1: Proximity to main road or railroad
* Condition2: Proximity to main road or railroad (if a second is present)
* BldgType: Type of dwelling
* HouseStyle: Style of dwelling
* OverallQual: Overall material and finish quality
* OverallCond: Overall condition rating
* YearBuilt: Original construction date
* YearRemodAdd: Remodel date
* RoofStyle: Type of roof
* RoofMatl: Roof material
* Exterior1st: Exterior covering on house
* Exterior2nd: Exterior covering on house (if more than one material)
* MasVnrType: Masonry veneer type
* MasVnrArea: Masonry veneer area in square feet
* ExterQual: Exterior material quality
* ExterCond: Present condition of the material on the exterior
* Foundation: Type of foundation
* BsmtQual: Height of the basement
* BsmtCond: General condition of the basement
* BsmtExposure: Walkout or garden level basement walls
* BsmtFinType1: Quality of basement finished area
* BsmtFinSF1: Type 1 finished square feet
* BsmtFinType2: Quality of second finished area (if present)
* BsmtFinSF2: Type 2 finished square feet
* BsmtUnfSF: Unfinished square feet of basement area
* TotalBsmtSF: Total square feet of basement area
* Heating: Type of heating
* HeatingQC: Heating quality and condition
* CentralAir: Central air conditioning
* Electrical: Electrical system
* 1stFlrSF: First Floor square feet
* 2ndFlrSF: Second floor square feet
* LowQualFinSF: Low quality finished square feet (all floors)
* GrLivArea: Above grade (ground) living area square feet
* BsmtFullBath: Basement full bathrooms
* BsmtHalfBath: Basement half bathrooms
* FullBath: Full bathrooms above grade
* HalfBath: Half baths above grade
* Bedroom: Number of bedrooms above basement level
* Kitchen: Number of kitchens
* KitchenQual: Kitchen quality
* TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
* Functional: Home functionality rating
* Fireplaces: Number of fireplaces
* FireplaceQu: Fireplace quality
* GarageType: Garage location
* GarageYrBlt: Year garage was built
* GarageFinish: Interior finish of the garage
* GarageCars: Size of garage in car capacity
* GarageArea: Size of garage in square feet
* GarageQual: Garage quality
* GarageCond: Garage condition
* PavedDrive: Paved driveway
* WoodDeckSF: Wood deck area in square feet
* OpenPorchSF: Open porch area in square feet
* EnclosedPorch: Enclosed porch area in square feet
* 3SsnPorch: Three season porch area in square feet
* ScreenPorch: Screen porch area in square feet
* PoolArea: Pool area in square feet
* PoolQC: Pool quality
* Fence: Fence quality
* MiscFeature: Miscellaneous feature not covered in other categories
* MiscVal: Value of miscellaneous feature
* MoSold: Month Sold
* YrSold: Year Sold
* SaleType: Type of sale
* SaleCondition: Condition of sale

# Imports

In [ ]:
# Data Processing
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)

# Data Visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='whitegrid')

# Modeling
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from math import sqrt

from sklearn.model_selection import RandomizedSearchCV

# Exploring the Data

In [ ]:
train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
sample_submission = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv")

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
test.head()

In [ ]:
test.shape

## Target Value: SalePrice

In [ ]:
plt.figure(figsize=(20,10))
b = sns.distplot(train['SalePrice'])
b.set_title("SalePrice Distribution");

In [ ]:
plt.figure(figsize=(20,10))
b = sns.boxplot(y = 'SalePrice', data = train)
b.set_title("SalePrice Distribution");

In [ ]:
len(train[train['SalePrice'] > 700000])

We have two outliers where `SalePrice` is `> 700000`. Let's get rid of these.

In [ ]:
train.shape

In [ ]:
train = train[train['SalePrice'] <= 700000]

In [ ]:
train.shape

# Handling missing values

**Where do we have NaN values?**

In [ ]:
train.columns[train.isna().any()].tolist()

In [ ]:
test.columns[test.isna().any()].tolist()

Let's look at which values have what percentage of missing data:

In [ ]:
#missing data
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

Let's delete the columns that have more than 15% missing data:

In [ ]:
train = train.drop(['PoolQC'], axis=1)
test = test.drop(['PoolQC'], axis=1)

train = train.drop(['MiscFeature'], axis=1)
test = test.drop(['MiscFeature'], axis=1)

train = train.drop(['Alley'], axis=1)
test = test.drop(['Alley'], axis=1)

train = train.drop(['Fence'], axis=1)
test = test.drop(['Fence'], axis=1)

train = train.drop(['FireplaceQu'], axis=1)
test = test.drop(['FireplaceQu'], axis=1)

train = train.drop(['LotFrontage'], axis=1)
test = test.drop(['LotFrontage'], axis=1)

Let's first fill all numeric values with the median.

In [ ]:
train = train.fillna(train.median())
test = test.fillna(test.median())

Fill the other values with `None`.

In [ ]:
test['MSZoning'] = test['MSZoning'].fillna('None')

In [ ]:
train = train.drop(['Utilities'], axis=1)
test = test.drop(['Utilities'], axis=1)

In [ ]:
test['Exterior1st'] = test['Exterior1st'].fillna('None')

train.loc[train['Exterior1st'].value_counts()[train['Exterior1st']].values < 18,'Exterior1st'] = 'Rare'
test.loc[test['Exterior1st'].value_counts()[test['Exterior1st']].values < 18,'Exterior1st'] = 'Rare'

In [ ]:
test['Exterior2nd'] = test['Exterior2nd'].fillna('None')

train.loc[train['Exterior2nd'].value_counts()[train['Exterior2nd']].values < 10,'Exterior2nd'] = 'Rare'
test.loc[test['Exterior2nd'].value_counts()[test['Exterior2nd']].values < 10,'Exterior2nd'] = 'Rare'

In [ ]:
train['MasVnrType'] = train['MasVnrType'].fillna('Missing')
test['MasVnrType'] = test['MasVnrType'].fillna('Missing')

In [ ]:
train['BsmtQual'] = train['BsmtQual'].fillna('None')
test['BsmtQual'] = test['BsmtQual'].fillna('None')

In [ ]:
train['BsmtCond'] = train['BsmtCond'].fillna('None')
test['BsmtCond'] = test['BsmtCond'].fillna('None')

In [ ]:
train['BsmtExposure'] = train['BsmtExposure'].fillna('None')
test['BsmtExposure'] = test['BsmtExposure'].fillna('None')

In [ ]:
train['BsmtFinType1'] = train['BsmtFinType1'].fillna('None')
test['BsmtFinType1'] = test['BsmtFinType1'].fillna('None')

In [ ]:
train['BsmtFinType2'] = train['BsmtFinType2'].fillna('None')
test['BsmtFinType2'] = test['BsmtFinType2'].fillna('None')

In [ ]:
train['Electrical'] = train['Electrical'].fillna('None')
test['Electrical'] = test['Electrical'].fillna('None')

In [ ]:
test['KitchenQual'] = test['KitchenQual'].fillna('None')

In [ ]:
test['Functional'] = test['Functional'].fillna('None')

In [ ]:
train['GarageType'] = train['GarageType'].fillna('None')
test['GarageType'] = test['GarageType'].fillna('None')

In [ ]:
train['GarageFinish'] = train['GarageFinish'].fillna('None')
test['GarageFinish'] = test['GarageFinish'].fillna('None')

In [ ]:
train['GarageQual'] = train['GarageQual'].fillna('None')
test['GarageQual'] = test['GarageQual'].fillna('None')

In [ ]:
train['GarageCond'] = train['GarageCond'].fillna('None')
test['GarageCond'] = test['GarageCond'].fillna('None')

In [ ]:
train['SaleType'] = train['SaleType'].fillna('None')
test['SaleType'] = test['SaleType'].fillna('None')

Let's check if we have any NaN values left:

In [ ]:
train.isna().all().sum()

In [ ]:
test.isna().all().sum()

Nope, we don't have any left!

# Feature Engineering

In [ ]:
y_train = train['SalePrice'].values
df = pd.concat((train, test)).reset_index(drop=True)
df.drop(['SalePrice'], axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
cols = ('BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(df[c].values)) 
    df[c] = lbl.transform(list(df[c].values))


In [ ]:
corrmat = train.corr()
plt.subplots(figsize=(12,9))
sns.heatmap(corrmat, vmax=0.9, square=True);

In [ ]:
df = pd.get_dummies(df)
print(df.shape)

# Modeling

In [ ]:
train = df[df['Id'] < 1461]
test = df[df['Id'] >= 1461]

In [ ]:
# Everything except target variable
X = train

# Target variable
y = y_train

In [ ]:
# Random seed for reproducibility
np.random.seed(42)

# Split into train & test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
# Put models in a dictionary
models = {"ElasticNet": ElasticNet(tol=0.1),
          "Lasso": Lasso(tol=0.1), 
          "BayesianRidge": BayesianRidge(n_iter=1000),
          "LassoLarsIC" : LassoLarsIC(max_iter=45),
          "RandomForestRegressor" : RandomForestRegressor(),
          "GradientBoostingRegressor" : GradientBoostingRegressor(),
          "XGBRegressor": XGBRegressor(),
          "LGBMRegressor": LGBMRegressor()
}

# Create function to fit and score models
def fit_and_score(models, X_train, X_test, y_train, y_test):
    """
    Fits and evaluates given machine learning models.
    models : a dict of different Scikit-Learn machine learning models
    X_train : training data
    X_test : testing data
    y_train : labels assosciated with training data
    y_test : labels assosciated with test data
    """
    # Random seed for reproducible results
    np.random.seed(42)
    # Make a list to keep model scores
    model_scores = {}
    # Loop through models
    for name, model in models.items():
        # Fit the model to the data
        model.fit(X_train, y_train)
        # Predicting target values
        y_pred = model.predict(X_test)
        # Evaluate the model and append its score to model_scores
        model_scores[name] = np.sqrt(mean_squared_error(y_test, y_pred))
    return model_scores

In [ ]:
model_scores = fit_and_score(models=models,
                             X_train=X_train,
                             X_test=X_test,
                             y_train=y_train,
                             y_test=y_test)
model_scores

In [ ]:
np.random.seed(42)

gbr = GradientBoostingRegressor(n_estimators=5000)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
# Plotting the Train and Test scores
plt.figure(figsize=(20,10))
plt.plot(y_test, label="True values")
plt.plot(y_pred, label="Predicted values")
plt.xticks(np.arange(1, 51, 1))
plt.xlabel("Object")
plt.ylabel("Values")
plt.legend();

# Predicting values

In [ ]:
y_pred = gbr.predict(test)

In [ ]:
sample_submission.head()

In [ ]:
sample_submission['SalePrice'] = y_pred
sample_submission.to_csv("submission.csv", index=False)
sample_submission.head()